<a href="https://colab.research.google.com/github/AP-047/RClass-Classification-by-Rational-Approximation/blob/main/notebooks/training_clb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set Locale to UTF-8**

In [ ]:
!apt-get install -y locales
!locale-gen en_US.UTF-8
!update-locale LANG=en_US.UTF-8

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
locales is already the newest version (2.35-0ubuntu3.8).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Generating locales (this might take a while)...
  en_US.UTF-8... done
Generation complete.


**Install CuPy for CUDA 12.2**

In [ ]:
!pip install cupy-cuda12x

**Step 1: Import Data**

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np

# Import MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
x_full = mnist.data.values  # Full dataset
y_full = mnist.target.values.astype(int)  # Labels (0–9)

print(f"x_full shape: {x_full.shape}, y_full shape: {y_full.shape}")

x_full shape: (70000, 784), y_full shape: (70000,)


**Step 2: Create Subset**

In [ ]:
# Create a subset of training data (1000 images per digit)
subset_size = 1000

x_subset = []
y_subset = []

for digit in range(10):
    digit_indices = np.where(y_full == digit)[0][:subset_size]
    x_subset.append(x_full[digit_indices])
    y_subset.append(y_full[digit_indices])

x_subset = np.vstack(x_subset)
y_subset = np.hstack(y_subset)

print(f"x_subset shape: {x_subset.shape}, y_subset shape: {y_subset.shape}")
print(f"Unique labels in y_subset: {np.unique(y_subset)}")
print(f"x_subset[3].shape: {x_subset[3].shape}")

x_subset shape: (10000, 784), y_subset shape: (10000,)
Unique labels in y_subset: [0 1 2 3 4 5 6 7 8 9]
x_subset[3].shape: (784,)


**Step 3: Preprocess Data**

In [ ]:
# Thresholding: Convert to binary
threshold_value = 128
x_binarized = (x_subset > threshold_value).astype(int)

print(f"x_binarized shape: {x_binarized.shape}")

x_binarized shape: (10000, 784)


**Step 4: Apply PCA**

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensionality
n_components = 180
  # Adjustable based on explained variance
pca = PCA(n_components=n_components)
x_pca = pca.fit_transform(x_binarized)

print(f"Original shape: {x_binarized.shape}, PCA shape: {x_pca.shape}")
print(f"variance retained: {np.sum(pca.explained_variance_ratio_)}")

print(f"shape of x_pca: ", x_pca.shape)
print(f"shape of x_pca[0]: ", x_pca[0].shape)

# print(f"heck any image vector: ", x_pca[250])

Original shape: (10000, 784), PCA shape: (10000, 180)
variance retained: 0.9005184150802336
shape of x_pca:  (10000, 180)
shape of x_pca[0]:  (180,)


**Step 5: Define Rational Function**

In [ ]:
import cupy as cp
def rational_function(x, alpha, beta):
    """
    r(x) = (α_0 + α_1*x1**1 + α_2*x2**2 + α_3*x3**3 + ...) /
            (β_0 + β_1*x1**1 + β_2*x2**2 + β_3*x3**3 + ...)
    """
    # Ensure x is treated as a 1D vector when passed for a single data point
    x = cp.asarray(x)  # Convert to CuPy array

    # Numerator polynomial
    numerator = alpha[0] + alpha[1] * x[0]**1 + alpha[2] * x[1]**2 + alpha[3] * x[2]**3 + alpha[4] * x[3]**4 + alpha[5] * x[4]**5 + alpha[6] * x[5]**6 + alpha[7] * x[6]**7 + alpha[8] * x[7]**8 + alpha[9] * x[8]**9 + alpha[10] * x[9]**10 + alpha[11] * x[10]**11 + alpha[12] * x[11]**12 + alpha[13] * x[12]**13 + alpha[14] * x[13]**14 + alpha[15] * x[14]**15 + alpha[16] * x[15]**16 + alpha[17] * x[16]**17 + alpha[18] * x[17]**18 + alpha[19] * x[18]**19 + alpha[20] * x[19]**20 + alpha[21] * x[20]**21 + alpha[22] * x[21]**22 + alpha[23] * x[22]**23 + alpha[24] * x[23]**24 + alpha[25] * x[24]**25 + alpha[26] * x[25]**26 + alpha[27] * x[26]**27 + alpha[28] * x[27]**28 + alpha[29] * x[28]**29 + alpha[30] * x[29]**30 + alpha[31] * x[30]**31 + alpha[32] * x[31]**32 + alpha[33] * x[32]**33 + alpha[34] * x[33]**34 + alpha[35] * x[34]**35 + alpha[36] * x[35]**36 + alpha[37] * x[36]**37 + alpha[38] * x[37]**38 + alpha[39] * x[38]**39 + alpha[40] * x[39]**40 + alpha[41] * x[40]**41 + alpha[42] * x[41]**42 + alpha[43] * x[42]**43 + alpha[44] * x[43]**44 + alpha[45] * x[44]**45

    # + alpha[46] * x[45]**46 + alpha[47] * x[46]**47 + alpha[48] * x[47]**48 + alpha[49] * x[48]**49 + alpha[50] * x[49]**50 + alpha[51] * x[50]**51 +

    # Denominator polynomial
    denominator = beta[0] + beta[1] * x[0]**1 + beta[2] * x[1]**2 + beta[3] * x[2]**3 + beta[4] * x[3]**4 + beta[5] * x[4]**5 + beta[6] * x[5]**6 + beta[7] * x[6]** + beta[8] * x[7]**8 + beta[9] * x[8]**9 + beta[10] * x[9]**10 + beta[11] * x[10]**11 + beta[12] * x[11]**12 + beta[13] * x[12]**13 + beta[14] * x[13]**14 + beta[15] * x[14]**15 + beta[16] * x[15]**16 + beta[17] * x[16]**17 + beta[18] * x[17]**18 + beta[19] * x[18]**19 + beta[20] * x[19]**20 + beta[21] * x[20]**21 + beta[22] * x[21]**22 + beta[23] * x[22]**23 + beta[24] * x[23]**24 + beta[25] * x[24]**25 + beta[26] * x[25]**26 + beta[27] * x[26]**27 + beta[28] * x[27]**28 + beta[29] * x[28]**29 + beta[30] * x[29]**30 + beta[31] * x[30]**31 + beta[32] * x[31]**32 + beta[33] * x[32]**33 + beta[34] * x[33]**34 + beta[35] * x[34]**35 + beta[36] * x[35]**36 + beta[37] * x[36]**37 + beta[38] * x[37]**38 + beta[39] * x[38]**39 + beta[40] + beta[40] * x[39]**40 + beta[41] * x[40]**41 + beta[42] * x[41]**42 + beta[43] * x[42]**43 + beta[44] * x[43]**44 + beta[45] * x[44]**45

    # + beta[46] * x[45]**46 + beta[47] * x[46]**47 + beta[48] * x[47]**48 + beta[49] * x[48]**49 + beta[50] * x[49]**50 + beta[51] * x[50]**51 + beta[52] * x[51]**52 + beta[53] * x[52]**53 + beta[54] * x[53]**54 + beta[55] * x[54]**55

    return numerator / denominator

# x = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# alpha = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# beta = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# print(f"lenght of x: {len(x)}")
# print(f"lenght of alpha: {len(alpha)}")
# print(f"lenght of beta: {len(beta)}")

# Result = rational_function(x, alpha, beta)
# print(f"Result: {Result}")

**Folder**

In [ ]:
import os

# Define the path to the models/ directory in Colab Notebooks
models_dir = "/content/drive/MyDrive/Colab Notebooks/models/"

# Create the directory if it does not exist
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Directory '{models_dir}' created successfully.")
else:
    print(f"Directory '{models_dir}' already exists.")

Directory '/content/drive/MyDrive/Colab Notebooks/models/' already exists.


**Step 6: Train Classifiers**

In [ ]:
import pickle
import cupy as cp
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt

# Feasibility check function
def check_feasibility_and_compute_coefficients(z, x_pca, y_binary):
    num_data_points = x_pca.shape[0]
    num_coefficients = n_components + 1  # (1 constant term + 4 variables)
    delta = 1e-6  # a small positive value

    # Construct G(x) and H(x) matrices for numerator and denominator
    G = cp.zeros((num_data_points, num_coefficients))  # Numerator matrix
    H = cp.zeros((num_data_points, num_coefficients))  # Denominator matrix

    for i in range(num_data_points):
      G[i, 0] = 1
      H[i, 0] = 1
      for j in range(num_coefficients-1):
        G[i, j+1] = x_pca[i, j] ** (j+1)
        H[i, j+1] = x_pca[i, j] ** (j+1)

    # print(f"G: {G}")
    # print(f"H: {H}")

    # Construct constraints for Ax <= b
    A = []
    b = []

    for i in range(num_data_points):
        f_plus_z = y_binary[i] + z  # Upper bound
        f_minus_z = y_binary[i] - z  # Lower bound

        # Constraint 1: (f(xi) - z) * β^T H(xi) - α^T G(xi) ≤ θ
        # (-G(xi))αT + (f(xi) - z).H(xi)βT + (-1)θ ≤ 0
        constraint_1 = cp.zeros(2 * num_coefficients + 1)
        # (1) Coefficients of α
        constraint_1[0:num_coefficients] = -G[i]
        # (2) Coefficients of β
        constraint_1[num_coefficients:2 * num_coefficients] = (f_minus_z) * H[i]
        # (3) Coefficient of θ (last element)
        constraint_1[-1] = -1
        A.append(constraint_1)
        b.append(0)

        # Constraint 2: α^T G(xi) + (-1).(f(xi) + z) * β^T H(xi) ≤ θ
        # G(xi).αT + (-1)(f(xi) - z).H(xi)βT + (-1)θ ≤ 0
        constraint_2 = cp.zeros(2 * num_coefficients + 1)
        # (1) Coefficients of α
        constraint_2[0:num_coefficients] = G[i]
        # (2) Coefficients of β
        constraint_2[num_coefficients:2 * num_coefficients] = -(f_plus_z) * H[i]
        # (3) Coefficient of θ (last element)
        constraint_2[-1] = -1
        A.append(constraint_2)
        b.append(0)

        # Constraint 3: β^T H(x) ≥ δ
        # (0)α^T + (-H(x)) β^T + (0)θ ≤ -δ
        constraint_3 = cp.zeros(2 * num_coefficients + 1)
        # Coefficient of β
        constraint_3[num_coefficients:2 * num_coefficients] = -H[i]
        A.append(constraint_3)
        b.append(-delta)

    # Convert CuPy arrays to NumPy arrays for SciPy
    A = cp.asnumpy(cp.array(A))
    b = cp.asnumpy(cp.array(b))

    # print(f"A =", len(A))
    # print(f"A: {A[0]}")
    # print(f"len(A[0]): {len(A[0])}")
    # print(f"len(b): {len(b)}")

    # Objective function to minimize θ
    c = cp.asnumpy(cp.zeros(2 * num_coefficients + 1))
    c[-1] = 1  # Only θ has a coefficient in the objective function

    # Solve the linear programming problem
    result = linprog(c, A_ub=A, b_ub=b, method="highs")

    # Check feasibility and return results
    if result.success:
        alpha_coefficients = result.x[:num_coefficients]
        beta_coefficients = result.x[num_coefficients:2 * num_coefficients]
        theta = result.x[-1]
        return True, alpha_coefficients, beta_coefficients, theta
    else:
        return False, None, None, None


# Bisection loop
def bisection_loop(x_pca, y_binary, uL, uH, precision):
    optimal_alpha, optimal_beta, optimal_theta = None, None, None
    z_values = []

    while uH - uL > precision:
        z = (uL + uH) / 2
        z_values.append(z)
        feasible, alpha_coefficients, beta_coefficients, theta = check_feasibility_and_compute_coefficients(z, x_pca, y_binary)

        if feasible:
            uH = z
            optimal_alpha, optimal_beta, optimal_theta = alpha_coefficients, beta_coefficients, theta
        else:
            uL = z

    return uH, optimal_alpha, optimal_beta, optimal_theta, z_values

# Train a classifier for each digit
for digit in range(10):
    print(f"Training classifier for digit {digit}...")

    # Assign labels: Positive for the current digit, negative for others
    y_binary = (y_subset == digit).astype(int)

    # Bisection parameters
    uL = 0  # Initial lower bound
    uH = 1000  # Initial upper bound
    precision = 1e-6  # Precision threshold

    # Run bisection loop
    optimal_z, optimal_alpha, optimal_beta, optimal_theta, z_values = bisection_loop(x_pca, y_binary, uL, uH, precision)

    # Print results
    print(f"Number of Iterations: {len(z_values)}")
    print(f"z Values in all Iterations: {z_values}")
    print(f"Optimal z (Maximum Deviation): {optimal_z}")

    # # Plot convergence of z values
    # plt.figure(figsize=(8, 6))
    # plt.plot(range(len(z_values)), z_values, marker='o', linestyle='-')
    # plt.xlabel("Iteration")
    # plt.ylabel("z Value")
    # plt.title("Convergence of z Values")
    # plt.grid(True)
    # plt.show()

    print(f"Optimized Coefficients (Numerator α): {optimal_alpha}")
    print(f"Optimized Coefficients (Denominator β): {optimal_beta}")
    print(f"Optimal θ: {optimal_theta}")
    # print(f"rational_function =", rational_function(x_pca[0], optimal_alpha, optimal_beta))

    # Save the model
    model = {
        "alpha": optimal_alpha,
        "beta": optimal_beta,
        "theta": optimal_theta,
        "n_components": n_components
    }

    with open(f"{models_dir}classifier_{digit}.pkl", "wb") as file:
        pickle.dump(model, file)

    print(f"Model for digit {digit} saved at {models_dir}")

Training classifier for digit 0...
Number of Iterations: 30
z Values in all Iterations: [500.0, 750.0, 875.0, 937.5, 968.75, 984.375, 992.1875, 996.09375, 998.046875, 999.0234375, 999.51171875, 999.755859375, 999.8779296875, 999.93896484375, 999.969482421875, 999.9847412109375, 999.9923706054688, 999.9961853027344, 999.9980926513672, 999.9990463256836, 999.9995231628418, 999.9997615814209, 999.9998807907104, 999.9999403953552, 999.9999701976776, 999.9999850988388, 999.9999925494194, 999.9999962747097, 999.9999981373549, 999.9999990686774]
Optimal z (Maximum Deviation): 1000
Optimized Coefficients (Numerator α): None
Optimized Coefficients (Denominator β): None
Optimal θ: None
Model for digit 0 saved at /content/drive/MyDrive/Colab Notebooks/models/
Training classifier for digit 1...


KeyboardInterrupt: 

++